In [182]:
import pandas as pd
from scipy.stats import zscore

In [192]:
spacy_features = pd.read_csv('linguistic_features/spacy_linguistic_features.csv', index_col = 0)
pd.set_option('display.max_columns', None)
spacy_features_finalized = spacy_features[
    ['Coherence', 
    'n_words', 
    'POS:NOUN', 
    'POS:PROPN', 
    'POS:PRON', 
    'POS:ADJ', 
    'POS:ADV', 
    'POS:CCONJ', 
    'POS:SCONJ', 
    'POS:DET', ''
    'open_closed_ratio', 
    'type_token_ratio', 
    'propositional_density', 
    'n_logical_operators', 
    'log10_freq_mean', 
    'semantic_diversity_mean', 
    'aoa_mean', 
    'concreteness_mean', 
    'semantic_thematic_distance', 
    'surprisal']
]
display(spacy_features_finalized)

,Coherence,n_words,POS:NOUN,POS:PROPN,POS:PRON,POS:ADJ,POS:ADV,POS:CCONJ,POS:SCONJ,POS:DET,open_closed_ratio,type_token_ratio,propositional_density,n_logical_operators,log10_freq_mean,semantic_diversity_mean,aoa_mean,concreteness_mean,semantic_thematic_distance,surprisal
0,3,210.0,0.141700,0.012146,0.165992,0.044534,0.076923,0.020243,0.024291,0.093117,0.820513,0.609524,0.356275,4.0,4.682792,2.107979,4.616221,2.597958,0.198185,4.895232
1,4,256.0,0.109272,0.033113,0.162252,0.043046,0.076159,0.026490,0.016556,0.076159,0.788732,0.535156,0.380795,7.0,4.666692,2.069585,4.606250,2.615833,0.195398,4.183853
2,4,758.0,0.139459,0.052973,0.103784,0.054054,0.055135,0.022703,0.028108,0.077838,0.879795,0.509235,0.375135,17.0,4.396347,2.102881,5.347357,2.420347,0.180549,5.146196
3,4,812.0,0.165787,0.024287,0.119324,0.071806,0.069694,0.030623,0.007392,0.080253,1.036082,0.492611,0.376980,30.0,4.421817,2.076899,5.135905,2.673849,0.178148,5.596125
4,5,155.0,0.177778,0.005556,0.150000,0.033333,0.033333,0.044444,0.033333,0.094444,0.878049,0.593548,0.338889,9.0,4.596490,2.064892,5.063814,2.591143,0.161879,4.702588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,3,206.0,0.165254,0.000000,0.144068,0.084746,0.072034,0.055085,0.038136,0.042373,1.067961,0.669903,0.453390,10.0,4.404882,2.118607,5.127013,2.415114,0.204966,3.565189
1535,4,96.0,0.111111,0.000000,0.138889,0.074074,0.129630,0.046296,0.037037,0.055556,1.000000,0.791667,0.481481,3.0,4.578900,2.106413,5.049481,2.350449,0.219363,3.443641
1536,3,101.0,0.214286,0.000000,0.107143,0.071429,0.044643,0.035714,0.035714,0.107143,1.061224,0.801980,0.419643,4.0,4.368046,2.086000,5.140270,2.389186,0.172722,4.055240
1537,5,131.0,0.138158,0.026316,0.151316,0.032895,0.052632,0.032895,0.019737,0.078947,0.830986,0.633588,0.368421,3.0,4.474590,2.176371,5.095055,2.484435,0.196328,3.690323


In [184]:
coherence = pd.read_csv('linguistic_features/coherence_results.csv')
grouped_df = coherence.groupby('pt', as_index=False).mean(numeric_only = True) # TODO: check that this is true!
local_coherence = grouped_df[['lc']].rename(columns = {'lc': 'local_coherence'})
display(local_coherence)

,local_coherence
0,0.274387
1,0.215685
2,0.161755
3,0.199047
4,0.375034
...,...
1534,0.268190
1535,0.323834
1536,0.295389
1537,0.258032


In [193]:
text_description = pd.read_csv('linguistic_features/text_description_ordered.csv', index_col = 0).drop('text', axis = 1)

# TODO: move text description in the repository

display(text_description)

,dependency_distance_mean,prop_adjacent_dependency_relation_mean,first_order_coherence,second_order_coherence,flesch_reading_ease
0,1.984713,0.442565,0.366634,0.316578,96.911667
1,2.568674,0.377021,0.310618,0.329741,93.589288
2,2.731996,0.413946,0.352635,0.331568,80.084680
3,2.332504,0.423813,0.308754,0.263125,89.716684
4,2.442240,0.426583,0.318063,0.399475,84.463050
...,...,...,...,...,...
1534,2.708896,0.422875,0.377143,0.412284,81.021902
1535,2.563026,0.405010,0.401308,0.352386,86.607500
1536,2.669339,0.444015,0.319754,0.358499,80.858078
1537,2.715251,0.364776,0.314997,0.391217,87.441264


In [186]:
syn_features = pd.read_csv('processed_data/syntactic_features.csv', index_col = 0)
syn_features

,coref_local,coref_global,n_constituents,n_noun_phrases,n_verb_phrases,n_sub_clauses,n_prep_phrases,mean_np_length,mean_vp_length
0,0.550000,0.433333,152.0,79.0,49.0,11.0,13.0,2.354430,4.632653
1,0.650000,0.638095,196.0,93.0,68.0,12.0,23.0,2.516129,7.161765
2,0.148148,0.096959,599.0,296.0,183.0,41.0,79.0,3.094595,7.650273
3,0.152941,0.105062,546.0,285.0,172.0,17.0,72.0,3.010526,6.168605
4,0.600000,0.545455,123.0,64.0,40.0,11.0,8.0,2.484375,6.775000
...,...,...,...,...,...,...,...,...,...
1534,0.461538,0.384615,153.0,70.0,55.0,14.0,14.0,2.871429,9.181818
1535,0.600000,0.400000,64.0,27.0,27.0,5.0,5.0,1.703704,6.481481
1536,0.166667,0.285714,70.0,33.0,20.0,5.0,12.0,2.424242,7.450000
1537,0.500000,0.600000,97.0,49.0,36.0,3.0,9.0,2.204082,5.694444


In [187]:
# TODO: fill this in with the other features
# TODO: make sure to normalize
# TODO: make sure that the phrase number is normalized y number of words

In [188]:
linguistic_feature_set = pd.concat([spacy_features_finalized, local_coherence, text_description, syn_features], axis = 1)
linguistic_feature_set = linguistic_feature_set.dropna()
linguistic_feature_set[['n_constituents', 'n_noun_phrases', 'n_verb_phrases', 'n_sub_clauses', 'n_prep_phrases']] = linguistic_feature_set[['n_constituents', 'n_noun_phrases', 'n_verb_phrases', 'n_sub_clauses', 'n_prep_phrases']].div(linguistic_feature_set['n_words'], axis = 0)
display(linguistic_feature_set)

,Coherence,n_words,POS:NOUN,POS:PROPN,POS:PRON,POS:ADJ,POS:ADV,POS:CCONJ,POS:SCONJ,POS:DET,open_closed_ratio,type_token_ratio,propositional_density,n_logical_operators,log10_freq_mean,semantic_diversity_mean,aoa_mean,concreteness_mean,semantic_thematic_distance,surprisal,local_coherence,dependency_distance_mean,prop_adjacent_dependency_relation_mean,first_order_coherence,second_order_coherence,flesch_reading_ease,coref_local,coref_global,n_constituents,n_noun_phrases,n_verb_phrases,n_sub_clauses,n_prep_phrases,mean_np_length,mean_vp_length
0,3,210.0,0.141700,0.012146,0.165992,0.044534,0.076923,0.020243,0.024291,0.093117,0.820513,0.609524,0.356275,4.0,4.682792,2.107979,4.616221,2.597958,0.198185,4.895232,0.274387,1.984713,0.442565,0.366634,0.316578,96.911667,0.550000,0.433333,0.723810,0.376190,0.233333,0.052381,0.061905,2.354430,4.632653
1,4,256.0,0.109272,0.033113,0.162252,0.043046,0.076159,0.026490,0.016556,0.076159,0.788732,0.535156,0.380795,7.0,4.666692,2.069585,4.606250,2.615833,0.195398,4.183853,0.215685,2.568674,0.377021,0.310618,0.329741,93.589288,0.650000,0.638095,0.765625,0.363281,0.265625,0.046875,0.089844,2.516129,7.161765
2,4,758.0,0.139459,0.052973,0.103784,0.054054,0.055135,0.022703,0.028108,0.077838,0.879795,0.509235,0.375135,17.0,4.396347,2.102881,5.347357,2.420347,0.180549,5.146196,0.161755,2.731996,0.413946,0.352635,0.331568,80.084680,0.148148,0.096959,0.790237,0.390501,0.241425,0.054090,0.104222,3.094595,7.650273
3,4,812.0,0.165787,0.024287,0.119324,0.071806,0.069694,0.030623,0.007392,0.080253,1.036082,0.492611,0.376980,30.0,4.421817,2.076899,5.135905,2.673849,0.178148,5.596125,0.199047,2.332504,0.423813,0.308754,0.263125,89.716684,0.152941,0.105062,0.672414,0.350985,0.211823,0.020936,0.088670,3.010526,6.168605
4,5,155.0,0.177778,0.005556,0.150000,0.033333,0.033333,0.044444,0.033333,0.094444,0.878049,0.593548,0.338889,9.0,4.596490,2.064892,5.063814,2.591143,0.161879,4.702588,0.375034,2.442240,0.426583,0.318063,0.399475,84.463050,0.600000,0.545455,0.793548,0.412903,0.258065,0.070968,0.051613,2.484375,6.775000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,3,206.0,0.165254,0.000000,0.144068,0.084746,0.072034,0.055085,0.038136,0.042373,1.067961,0.669903,0.453390,10.0,4.404882,2.118607,5.127013,2.415114,0.204966,3.565189,0.268190,2.708896,0.422875,0.377143,0.412284,81.021902,0.461538,0.384615,0.742718,0.339806,0.266990,0.067961,0.067961,2.871429,9.181818
1535,4,96.0,0.111111,0.000000,0.138889,0.074074,0.129630,0.046296,0.037037,0.055556,1.000000,0.791667,0.481481,3.0,4.578900,2.106413,5.049481,2.350449,0.219363,3.443641,0.323834,2.563026,0.405010,0.401308,0.352386,86.607500,0.600000,0.400000,0.666667,0.281250,0.281250,0.052083,0.052083,1.703704,6.481481
1536,3,101.0,0.214286,0.000000,0.107143,0.071429,0.044643,0.035714,0.035714,0.107143,1.061224,0.801980,0.419643,4.0,4.368046,2.086000,5.140270,2.389186,0.172722,4.055240,0.295389,2.669339,0.444015,0.319754,0.358499,80.858078,0.166667,0.285714,0.693069,0.326733,0.198020,0.049505,0.118812,2.424242,7.450000
1537,5,131.0,0.138158,0.026316,0.151316,0.032895,0.052632,0.032895,0.019737,0.078947,0.830986,0.633588,0.368421,3.0,4.474590,2.176371,5.095055,2.484435,0.196328,3.690323,0.258032,2.715251,0.364776,0.314997,0.391217,87.441264,0.500000,0.600000,0.740458,0.374046,0.274809,0.022901,0.068702,2.204082,5.694444


In [189]:
coherence_df = linguistic_feature_set['Coherence']
linguistic_feature_set_norm = linguistic_feature_set.drop('Coherence', axis = 1)
linguistic_feature_set_norm = linguistic_feature_set_norm.apply(zscore)
linguistic_feature_set_final = pd.concat([coherence_df, linguistic_feature_set_norm], axis = 1)
linguistic_feature_set_final

,Coherence,n_words,POS:NOUN,POS:PROPN,POS:PRON,POS:ADJ,POS:ADV,POS:CCONJ,POS:SCONJ,POS:DET,open_closed_ratio,type_token_ratio,propositional_density,n_logical_operators,log10_freq_mean,semantic_diversity_mean,aoa_mean,concreteness_mean,semantic_thematic_distance,surprisal,local_coherence,dependency_distance_mean,prop_adjacent_dependency_relation_mean,first_order_coherence,second_order_coherence,flesch_reading_ease,coref_local,coref_global,n_constituents,n_noun_phrases,n_verb_phrases,n_sub_clauses,n_prep_phrases,mean_np_length,mean_vp_length
0,3,0.031521,0.007830,-0.282531,0.624773,-0.383468,1.048965,-0.641477,0.204631,0.549315,0.007916,0.012986,-0.134134,-0.459735,0.230932,0.281715,-0.538893,0.090647,0.109211,0.759521,-0.262331,-1.087732,0.369707,-0.390776,-0.751544,0.497214,0.538403,0.276915,-0.555212,0.404909,-0.498805,0.284006,-0.891422,-0.328190,-0.997350
1,4,0.371692,-0.719002,0.410407,0.537476,-0.445036,1.019417,-0.209887,-0.342357,-0.023866,-0.154417,-0.555387,0.228985,0.086386,0.161142,-0.545229,-0.563719,0.204741,0.010791,-0.213751,-0.761177,0.514575,-1.412746,-0.927500,-0.630440,0.225059,0.915415,1.115458,0.105653,0.094349,0.031449,0.030278,0.006135,-0.162578,-0.006094
2,4,4.083993,-0.042397,1.066779,-0.827151,0.010404,0.206455,-0.471540,0.474520,0.032879,0.310729,-0.753499,0.145170,1.906788,-1.010760,0.171913,1.281530,-1.043011,-0.513721,1.102876,-1.219473,0.962705,-0.408579,-0.524902,-0.613629,-0.881179,-0.976627,-1.100609,0.494636,0.749187,-0.365937,0.362749,0.468033,0.429886,0.185372
3,4,4.483324,0.547678,0.118736,-0.464441,0.744873,0.769418,0.075642,-0.990419,0.114524,1.109038,-0.880552,0.172490,4.273311,-0.900353,-0.387705,0.755045,0.575046,-0.598518,1.718446,-0.902561,-0.133443,-0.140238,-0.945360,-1.243337,-0.092167,-0.958557,-1.067427,-1.367487,-0.201459,-0.852027,-1.165053,-0.031574,0.343783,-0.395351
4,5,-0.375206,0.816435,-0.500332,0.251526,-0.846909,-0.636591,1.030509,0.844017,0.594168,0.301808,-0.109110,-0.391618,0.450466,-0.143172,-0.646314,0.575548,0.047146,-1.173180,0.495955,0.592963,0.167656,-0.064916,-0.856162,0.011139,-0.522522,0.726909,0.736075,0.546963,1.288115,-0.092700,1.140528,-1.222055,-0.195101,-0.157682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,3,0.001940,0.535744,-0.683940,0.113069,1.280264,0.859907,1.765604,1.183605,-1.165806,1.271873,0.474449,1.304080,0.632507,-0.973763,0.510633,0.732906,-1.076414,0.348739,-1.060174,-0.314994,0.899323,-0.165763,-0.290076,0.128986,-0.804405,0.204892,0.077405,-0.256370,-0.470404,0.053868,1.001978,-0.696857,0.201319,0.785644
1535,4,-0.811512,-0.677771,-0.683940,-0.007805,0.838726,3.087055,1.158446,1.105922,-0.720243,0.924730,1.405061,1.720102,-0.641775,-0.219422,0.247992,0.539861,-1.489153,0.857275,-1.226470,0.157871,0.499077,-0.651595,-0.058541,-0.422100,-0.346858,0.726909,0.140408,-1.458316,-1.879093,0.288023,0.270291,-1.206942,-0.994663,-0.272723
1536,3,-0.774537,1.634691,-0.683940,-0.748751,0.729269,-0.199267,0.427377,1.012384,1.023363,1.237463,1.483884,0.804305,-0.459735,-1.133441,-0.191677,0.765914,-1.241905,-0.790163,-0.389713,-0.083855,0.790783,0.409152,-0.839955,-0.365860,-0.817825,-0.906810,-0.327616,-1.041040,-0.784908,-1.078681,0.151473,0.936754,-0.256688,0.106877
1537,5,-0.552686,-0.071569,0.185778,0.282236,-0.865056,0.109646,0.232586,-0.117447,0.070380,0.061412,0.196901,0.045738,-0.641775,-0.671588,1.754793,0.653335,-0.633951,0.043630,-0.888973,-0.401311,0.916759,-1.745756,-0.885538,-0.064838,-0.278560,0.349897,0.959450,-0.292094,0.353314,0.182260,-1.074511,-0.673048,-0.482177,-0.581193


In [190]:
linguistic_feature_set_final.to_csv('linguistic_features/linguistic_feature_set.csv')